In [3]:
!pip install pandas sklearn nltk imblearn fasttext

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296184 sha256=27eed9a9817a1f37d26fadc940ff95890ed2a7160fe4cd73b706dd2d2d3d363c
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
# Importing necessary libraries
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from imblearn.over_sampling import SMOTE
import numpy as np

# Load dataset
df = pd.read_csv('/content/dataset.csv')  # Adjust path to your dataset

# Assuming the class labels are in the 'label' column
class_distribution = df['label'].value_counts()

# Display the class distribution
print("Class distribution in the dataset:")
print(class_distribution)

# If you want to display the class names along with their counts
for label, count in class_distribution.items():
    print(f"Class '{label}': {count} samples")

# Bangla and Banglish stopwords
bangla_stopwords = set(stopwords.words('bengali'))
banglish_stopwords = set(stopwords.words('english'))

# Cleaning text function
def clean_text(text, language='banglish'):
    # Lowercasing
    text = text.lower()

    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)

    # Remove stopwords
    if language == 'bangla':
        tokens = word_tokenize(text)
        text = ' '.join([word for word in tokens if word not in bangla_stopwords])
    elif language == 'banglish':
        tokens = word_tokenize(text)
        text = ' '.join([word for word in tokens if word not in banglish_stopwords])

    return text

# Applying cleaning function
df['clean_bangla'] = df['bangla'].apply(lambda x: clean_text(x, language='bangla'))
df['clean_banglish'] = df['banglish'].apply(lambda x: clean_text(x, language='banglish'))

# Convert categorical labels to numerical labels
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Check class distribution before SMOTE
print("Class distribution before SMOTE:")
print(df['label_encoded'].value_counts())

# TF-IDF Vectorization (you can use for both bangla and banglish text separately)
tfidf_vectorizer_bangla = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
tfidf_vectorizer_banglish = TfidfVectorizer(max_features=5000)

# Vectorize the bangla and banglish columns
X_bangla_tfidf = tfidf_vectorizer_bangla.fit_transform(df['clean_bangla']).toarray()
X_banglish_tfidf = tfidf_vectorizer_banglish.fit_transform(df['clean_banglish']).toarray()

# Combining the two vectors
X_combined = np.hstack((X_bangla_tfidf, X_banglish_tfidf))

# Labels (y)
y = df['label_encoded']

# Applying SMOTE for balancing the dataset
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_combined, y)

# Check class distribution after SMOTE
print("\nClass distribution after SMOTE:")
unique, counts = np.unique(y_res, return_counts=True)
print(dict(zip(unique, counts)))

# Train Test Split (with balanced data)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Display shapes to confirm
print("\nTraining data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

# Now you can use X_train, X_test, y_train, and y_test for training your model.


Class distribution in the dataset:
label
Personal          4225
Geopolitical      3761
Religious         1714
Political         1596
Neutral            835
Gender abusive     316
Name: count, dtype: int64
Class 'Personal': 4225 samples
Class 'Geopolitical': 3761 samples
Class 'Religious': 1714 samples
Class 'Political': 1596 samples
Class 'Neutral': 835 samples
Class 'Gender abusive': 316 samples
Class distribution before SMOTE:
label_encoded
3    4225
1    3761
5    1714
4    1596
2     835
0     316
Name: count, dtype: int64

Class distribution after SMOTE:
{0: 4225, 1: 4225, 2: 4225, 3: 4225, 4: 4225, 5: 4225}

Training data shape: (20280, 10000)
Test data shape: (5070, 10000)


In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Train Naive Bayes Classifier
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Predictions and accuracy
y_pred_nb = nb_model.predict(X_test)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"Naive Bayes Accuracy: {accuracy_nb * 100:.2f}%")


Naive Bayes Accuracy: 84.95%


In [10]:
# Import necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score

# Tokenize the text data
tokenizer = Tokenizer(num_words=10000)  # Adjust num_words based on your dataset size
tokenizer.fit_on_texts(df['clean_bangla'].values)  # Tokenize on the clean Bangla text (or combined text if needed)

# Convert text to sequences
X_bangla_seq = tokenizer.texts_to_sequences(df['clean_bangla'].values)
X_banglish_seq = tokenizer.texts_to_sequences(df['clean_banglish'].values)

# Padding the sequences to ensure uniform input length
X_bangla_seq_padded = pad_sequences(X_bangla_seq, maxlen=100)  # Adjust maxlen based on text length
X_banglish_seq_padded = pad_sequences(X_banglish_seq, maxlen=100)

# Combining Bangla and Banglish padded sequences
X_combined_seq = np.hstack((X_bangla_seq_padded, X_banglish_seq_padded))

# Splitting the data into train and test sets
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(X_combined_seq, df['label_encoded'], test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=X_train_seq.shape[1]))  # Embedding layer
model.add(LSTM(128, return_sequences=False))  # LSTM layer
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))  # Dense layer
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))  # Output layer with 4 classes

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the LSTM model
model.fit(X_train_seq, y_train_seq, epochs=5, batch_size=32, validation_data=(X_test_seq, y_test_seq))

# Evaluate the model
score, accuracy = model.evaluate(X_test_seq, y_test_seq)
print(f"LSTM Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/pyt